In [ ]:
# Copyright 2025 Forusone

# Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
#     https://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

## Agent Development Kit - Session


### Install adk

In [1]:
%pip install --upgrade --quiet google-adk

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 8.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 232.1/232.1 kB 18.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 95.2/95.2 kB 8.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.6/7.6 MB 97.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 217.1/217.1 kB 17.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 334.1/334.1 kB 25.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 165.9/165.9 kB 14.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 76.1/76.1 kB 5.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 65.3/65.3 kB 5.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 119.0/119.0 kB 10.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 188.4/188.4 kB 16.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.5/62.5 kB 5.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 100.9

In [3]:
# @title Project Information

PROJECT_ID = "ai-hangsik"
LOCATION = "us-central1"

MODEL_LIST = ["gemini-2.0-flash", \
              "gemini-2.5-pro-preview-03-25", \
              "gemini-2.5-flash-preview-04-17"]

MODEL_NAME = MODEL_LIST[2]
print(MODEL_NAME)

gemini-2.5-flash-preview-04-17


In [4]:
# @title Authentication to access to GCP
import sys

if "google.colab" in sys.modules:
    from google.colab import auth
    auth.authenticate_user(project_id = PROJECT_ID)

### Initialize Vertex AI

* Please refer to configuration to **initialize Vertex AI : [Link](https://google.github.io/adk-docs/get-started/quickstart/#set-up-the-model)**
* You can set up api key rather than usning Vertex AI.

In [5]:
import os

GOOGLE_GENAI_USE_VERTEXAI = "TRUE"                            #@param {type:"string"}
os.environ["GOOGLE_GENAI_USE_VERTEXAI"] = GOOGLE_GENAI_USE_VERTEXAI

GOOGLE_CLOUD_PROJECT = "ai-hangsik"                           #@param {type:"string"}
os.environ["GOOGLE_CLOUD_PROJECT"] = GOOGLE_CLOUD_PROJECT

GOOGLE_CLOUD_LOCATION = "us-central1"                         #@param {type:"string"}
os.environ["GOOGLE_CLOUD_LOCATION"] = GOOGLE_CLOUD_LOCATION


## Session store

In [6]:
import sqlite3
from google.adk.sessions import DatabaseSessionService
from google.adk.sessions import InMemorySessionService, Session

APP_NAME = "Search_Assistant"
USER_ID = "shins"

# https://google.github.io/adk-docs/sessions/

if True: # InMemory
  session_service = InMemorySessionService()

else: # SQLite3
  db_url = "sqlite:///./mydatabase.db"
  session_service = DatabaseSessionService(db_url=db_url)

In [9]:
#@title Build a Agent with adk.
from google.adk.agents import LlmAgent
from google.adk.tools import google_search
from google.genai import types

PROMPT = """

You are a helpful assistant. Answer user questions using Google Search when needed.
Wirte a simple and clear answer to the user question.

"""

# https://googleapis.github.io/python-genai/genai.html#genai.types.GenerateContentConfig
generate_content_config= types.GenerateContentConfig(
  temperature=0.2,
  max_output_tokens=250,
)

# https://google.github.io/adk-docs/api-reference/google-adk.html#google.adk.agents.LlmAgent
agent = LlmAgent(
      name="Search_assistant",
      model=MODEL_NAME,
      instruction=PROMPT,
      generate_content_config = generate_content_config,
      description="An assistant that can search the web.",
      tools=[google_search] # Use Google search tool
)

In [10]:
from google.adk.sessions import InMemorySessionService, Session

APP_NAME = "Search_Assistant"
USER_ID = "shins"

# https://google.github.io/adk-docs/sessions/
session_service = InMemorySessionService()
session = session_service.create_session(app_name=APP_NAME,
                                         user_id=USER_ID,
                                         state={"initial_key": "initial_value"})

In [ ]:
from google.adk.runners import Runner

def call_agent(app_name:str,
               user_id:str,
               session_id:str,
               query:str,):

  from google.adk.runners import Runner


  app_name = "Search_Assistant"

  # Runner : An in-memory Runner for testing and development.
  # https://google.github.io/adk-docs/api-reference/google-adk.html#google.adk.runners.Runner
  runner = Runner(agent=agent,
                  app_name=app_name,
                  session_service=session_service)

  # https://googleapis.github.io/python-genai/genai.html#genai.types.Content
  content = types.Content(role='user', parts=[types.Part(text=query)])

  # https://google.github.io/adk-docs/api-reference/google-adk.html#google.adk.runners.Runner.run
  events = runner.run(user_id=user_id,
                      session_id=session_id,
                      new_message=content)

  for event in events:
      if event.is_final_response():
          final_response = event.content.parts[0].text
          #print("Agent Response: \n", final_response)
          return final_response
      else:
          return "No results"


In [ ]:
QUERY = "What is the exchange rate from US dollars to Korean currency recently ?"
# QUERY = "What is the situation of the world economy?"

call_agent(app_name = session.app_name,
           user_id = session.user_id,
           session_id = session.id,
           query=QUERY
)


"Based on recent information, the exchange rate from US Dollars (USD) to South Korean Won (KRW) is approximately 1 USD to 1437-1438 KRW.\n\nFor example, one source shows the rate as 1 USD = 1,437.42 KRW. Another source indicates the rate was 1,438.49 KRW on Friday, April 25th.\n\nKeep in mind that exchange rates can change frequently. For the most current rate, it's best to use a live currency converter."

In [ ]:
session_service.list_sessions(app_name=APP_NAME, user_id=USER_ID)

ListSessionsResponse(sessions=[Session(id='6c14ca13-43d9-41f4-94ff-f62d9cbe4731', app_name='Search_Assistant', user_id='shins', state={}, events=[], last_update_time=1745747742.619486)])

In [ ]:
from pprint import pprint

pprint(session_service.sessions[session.app_name][session.user_id][session.id].events)


[Event(content=Content(parts=[Part(video_metadata=None, thought=None, code_execution_result=None, executable_code=None, file_data=None, function_call=None, function_response=None, inline_data=None, text='What is the exchange rate from US dollars to Korean currency recently ?')], role='user'), grounding_metadata=None, partial=None, turn_complete=None, error_code=None, error_message=None, interrupted=None, custom_metadata=None, invocation_id='e-aaeeaa15-6cac-4624-9ccf-2f38208bce1b', author='user', actions=EventActions(skip_summarization=None, state_delta={}, artifact_delta={}, transfer_to_agent=None, escalate=None, requested_auth_configs={}), long_running_tool_ids=None, branch=None, id='a4FQx2xF', timestamp=1745747737.130283),
 Event(content=Content(parts=[Part(video_metadata=None, thought=None, code_execution_result=None, executable_code=None, file_data=None, function_call=None, function_response=None, inline_data=None, text="As of recently, the exchange rate from US Dollars (USD) to So

In [ ]:
from google.adk.sessions import InMemorySessionService, Session

print(f"--- Examining Session Properties ---")
print(f"ID (`id`):                {session.id}")
print(f"Application Name (`app_name`): {session.app_name}")
print(f"User ID (`user_id`):         {session.user_id}")
print(f"State (`state`):           {session.state}") # Note: Only shows initial state here
print(f"Events (`events`):         {session.events}") # Initially empty
print(f"Last Update (`last_update_time`): {session.last_update_time:.2f}")
print(f"---------------------------------")


--- Examining Session Properties ---
ID (`id`):                6c14ca13-43d9-41f4-94ff-f62d9cbe4731
Application Name (`app_name`): Search_Assistant
User ID (`user_id`):         shins
State (`state`):           {'initial_key': 'initial_value'}
Events (`events`):         []
Last Update (`last_update_time`): 1745747733.31
---------------------------------


In [ ]:
# Clean up (optional for this example)
session_service.delete_session(app_name=session.app_name,
                               user_id=session.user_id,
                               session_id=session.id)

## End of notebook